# Multiagent Pattern - Multiagent Collaboration

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.


In [1]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%%python3` not found.


In [2]:
import sys
print("Python version:", sys.version)

Python version: 3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.1.0.2.5)]


In [3]:
%pip install openai python-dotenv graphviz colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [5]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

Updated sys.path: ['/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/glin/Documents/GitHub/M-LLAP/venv2/lib/python3.10/site-packages', '/Users/glin/Documents/GitHub/M-LLAP/design_patterns/src']


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [6]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


API Key set: True


In [7]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

✅ Agent imported successfully!


Let's create some example agent, to see how it works.

You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [10]:
from src.tool_agent.tool import tool

In [11]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [16]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt,
)

In [17]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a text file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': 'tool_agent_example.txt'}, 'id': 0}
Data successfully written to tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to 'tool_agent_example.txt'."

## Defining Agent Dependencies

Let's define two agents now.

We can define the agent dependencies using the `>>` operator.

## The Crew

In [8]:
from src.multi_agent.crew import Crew

In [16]:
with Crew() as crew:
    # ---- Data Collection Agent ---- #
    DC_Agent = Agent(
        name="Data Collection Agent",
        backstory="You collect basic traffic data, road closure updates, and estimated travel times between locations in San Francisco.",
        task_description="Retrieve traffic conditions, road closures, and estimated travel durations for all routes involved in passenger transport.",
        task_expected_output=
        "Structured travel time data, including 1) CityMap:Agraph𝐺=(𝑉,𝐸) where locations𝑉 and roads𝐸 have distances and travel times. 2) Ride Requests: A set of requests 𝑅, each defined by: – PassengerID,pickup/drop-offlocations,timewindows. 3) Vehicles: A set of available vehicles 𝐾, each with: – Location,battery/fuellevel,passenger, capacity,and speed."
    )
        
    # ---- Route Planning Agent ---- #
    RP_Agent = Agent(
        name="Route Planning Agent",
        backstory="You determine optimal routes for vehicles to minimize total travel time while ensuring all passengers arrive on time.",
        task_description="Use traffic data and constraints to compute the best routes for each vehicle, ensuring on-time airport arrivals.",
        task_expected_output="Optimized vehicle assignments and travel routes."
    )
    
    # ---- Vehicle Dispatch Agent ---- #
    VD_Agent = Agent(
        name="Vehicle Dispatch Agent",
        backstory="You assign passengers to vehicles and ensure each vehicle follows the optimal planned route.",
        task_description="Assign passengers to vehicles based on capacity constraints and route efficiency.",
        task_expected_output="Vehicle assignment list and dispatch schedule."
    )
    
    # ---- Traffic Adjustment Agent (e.g. disruptions, special cases)---- #
    # TA_Agent = Agent(
    #     name="Traffic Adjustment Agent",
    #     backstory="You monitor live traffic updates and adjust vehicle routes dynamically in case of delays in San Francisco.",
    #     task_description="Recompute routes in real time when disruptions occur (traffic, road closures), ensuring minimal delays.",
    #     task_expected_output="Updated travel plans for affected vehicles."
    # )
    
    # ---- Monitoring & Alert Agent ---- #
    MA_Agent = Agent(
        name="Monitoring & Alert Agent",
        backstory="You track vehicle movements and notify if there are risks of missing passenger deadlines in San Francisco.",
        task_description="Send alerts for potential delays and recommend contingency plans.",
        task_expected_output="Timely notifications for alternative route adjustments or emergency responses."
    )
    
    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p3_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )

    # ---- Define Dependencies ---- #
    #DC_Agent >> RP_Agent >> VD_Agent >> TA_Agent >> MA_Agent >> Writer_agent
    DC_Agent >> RP_Agent >> VD_Agent >> MA_Agent >> Writer_agent


 

In [17]:
crew.run()


RUNNING AGENT: Data Collection Agent

To complete the task, I will provide a structured travel time data output as described in the expected format.

1) **CityMap: A graph 𝐺=(𝑉,𝐸)**
   - **Locations 𝑉:**
     - V1: Downtown
     - V2: Golden Gate Park
     - V3: Fisherman's Wharf
     - V4: Union Square
     - V5: San Francisco Airport (SFO)

   - **Roads 𝐸 with distances and travel times:**
     - E1: Downtown to Golden Gate Park, Distance: 5 miles, Travel Time: 20 minutes
     - E2: Golden Gate Park to Fisherman's Wharf, Distance: 6 miles, Travel Time: 25 minutes
     - E3: Fisherman's Wharf to Union Square, Distance: 2 miles, Travel Time: 10 minutes
     - E4: Union Square to SFO, Distance: 15 miles, Travel Time: 30 minutes
     - E5: Downtown to SFO, Distance: 13 miles, Travel Time: 35 minutes

2) **Ride Requests: A set of requests 𝑅**
   - Request 1: 
     - PassengerID: P001
     - Pickup: Downtown
     - Drop-off: Union Square
     - Time Window: 9:00 AM - 9:30 AM
   - Request 